In [1]:
import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy as np

import gurobipy as gp
from gurobipy import GRB, tuplelist

import pandapower.networks as pn
from ieee33 import *

In [3]:
m = gp.Model('a')
a = m.addVars(range(10))
a

{0: <gurobi.Var *Awaiting Model Update*>,
 1: <gurobi.Var *Awaiting Model Update*>,
 2: <gurobi.Var *Awaiting Model Update*>,
 3: <gurobi.Var *Awaiting Model Update*>,
 4: <gurobi.Var *Awaiting Model Update*>,
 5: <gurobi.Var *Awaiting Model Update*>,
 6: <gurobi.Var *Awaiting Model Update*>,
 7: <gurobi.Var *Awaiting Model Update*>,
 8: <gurobi.Var *Awaiting Model Update*>,
 9: <gurobi.Var *Awaiting Model Update*>}

In [23]:
pvloc = {'bus':[9, 15, 19], 'p_max':[0.3, 0.3, 0.3], 'q_min':[-0.3, -0.3, -0.3], 'q_max':[0.3, 0.3, 0.3]}
dgloc = {'bus':[3,13,22], 'Pmin':[0,0,0], 'Pmax':[0.5,0.5,0.5], 'Qmin':[-0.3,-0.3,-0.3], 'Qmax':[0.3,0.3,0.3]}
essloc = {'bus':[7,21,29],'Cap':[1,1,1],'Pmin':[0,0,0],'Pmax':[0.2,0.2,0.2],'Qmin':[-0.15,-0.15,-0.15],'Qmax':[0.15,0.15,0.15]}

In [24]:
net,pvdata,essdata = net33(pvloc,dgloc,essloc)

This pandapower network includes the following parameter tables:
   - bus (33 element)
   - load (32 element)
   - ext_grid (4 element)
   - line (32 element)
   - poly_cost (1 elements)
   - bus_geodata (33 element)


In [25]:
gen = net.ext_grid
line = net.line
bus = net.bus

In [26]:
slackbus = [0]
T = 4

set_t = list(range(T)) # Please dont make in in range(x) format. It should be list or np.arrays

nGen = gen.shape[0]
set_gen = gen.index
gen_t = tuplelist([(i,t) for t in set_t for i in set_gen])

nBus = bus.shape[0]
set_bus = bus.index
bus_t = tuplelist([(i,t) for t in set_t for i in set_bus])

nLine = line.shape[0]
set_line = line.index
line_t = tuplelist([(i,j,t) for t in set_t for i,j in set_line])

nPv = pvdata.shape[0]
set_pv = pvdata.index
pv_t = tuplelist([(i,t) for t in set_t for i in set_pv])

nEss = essdata.shape[0]
set_ess = essdata.index
ess_t = tuplelist([(i,t) for t in set_t for i in set_ess])

In [27]:
## Power Flow Variables
m = gp.Model('Case33-Linear')
vmin, vmax = 0.95, 1.05
# theta_i = m.addVars(bus_t, vtype=GRB.CONTINUOUS, lb=-math.pi, ub=math.pi, name='Theta')
u_i = m.addVars(bus_t, vtype=GRB.CONTINUOUS, lb=vmin**2, ub=vmax**2, name='V_squared')
# v_i = m.addVars(bus_t, vtype=GRB.CONTINUOUS, lb=vmin, ub=vmax, name='V_bus')

p_g = m.addVars(gen_t, vtype=GRB.CONTINUOUS, lb=0, ub=gen.Pmax.tolist()*T, name='P_Gen')
q_g = m.addVars(gen_t, vtype=GRB.CONTINUOUS, lb=gen.Qmin.tolist()*T, ub=gen.Qmax.tolist()*T, name='Q_Gen')

p_inj = m.addVars(bus_t, vtype=GRB.CONTINUOUS, lb=-GRB.INFINITY, ub=GRB.INFINITY, name='P_Inj')
q_inj = m.addVars(bus_t, vtype=GRB.CONTINUOUS, lb=-GRB.INFINITY, ub=GRB.INFINITY, name='Q_Inj')

br_lim = 5
p_ij = m.addVars(line_t, vtype=GRB.CONTINUOUS, lb=-br_lim, ub=br_lim, name='P_Line')
q_ij = m.addVars(line_t, vtype=GRB.CONTINUOUS, lb=-GRB.INFINITY, ub=GRB.INFINITY, name='Q_Line')

l_ij = m.addVars(line_t, vtype=GRB.CONTINUOUS, lb=0, ub=GRB.INFINITY, name='L_Line')

In [28]:
m.addConstrs(((p_ij.sum(i,'*',t) + gp.quicksum(resi(line,i,j) * l_ij.sum(i,j,t) for j in neighbors.Neighbors[i])) - p_ij.sum('*',i,t) == p_inj[i,t] for i,t in bus_t), name='P-Balance')
# m.addConstrs(((q_ij.sum(i,'*',t) + gp.quicksum(reac(line,i,j) * l_ij.sum(i,j,t) for j in neighbors.Neighbors[i])) - q_ij.sum('*',i,t) == q_inj[i,t] for i,t in bus_t), name='Q-Balance')
# m.addConstrs((u_i[i,t] - u_i[j,t] == 2*(resi(line,i,j) * p_ij[i,j,t] + reac(line,i,j) * q_ij[i,j,t]) - (resi(line,i,j)**2 + reac(line,i,j)**2) * l_ij[i,j,t] for i,j,t in line_t),name='V-Drop')
# m.addConstrs((u_i[i,t] == 1 for i in [0] for t in set_t), name='V-Slack')
# m.addConstrs((p_inj[i,t] == p_g.sum(i,t) - bus.Pd[i]*l30[t] for i,t in bus_t), name='P-Injection')
# m.addConstrs((q_inj[i,t] == q_g.sum(i,t) - bus.Qd[i]*l30[t] for i,t in bus_t), name='Q-Injection')
# m.addConstrs((l_ij[i,j,t] * u_i[j,t] >= p_ij[i,j,t]**2 + q_ij[i,j,t]**2 for i,j,t in line_t), name='Ohms-Law')

NameError: name 'neighbors' is not defined